In [1]:
import cv2
import numpy as np

In [2]:
img1 = cv2.imread('./img/taekwonv1.jpg')
img2 = cv2.imread('./img/figures.jpg')
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

In [4]:
# ORB, BF-Mamming로 knnMatch
detector = cv2.ORB_create()
kp1, desc1 = detector.detectAndCompute(gray1, None)
kp2, desc2 = detector.detectAndCompute(gray2, None)
matcher = cv2.BFMatcher(cv2.NORM_HAMMING2)
matches = matcher.knnMatch(desc1, desc2, 2)

In [8]:
# 이웃 거리의 75%로 좋은 매칭점 추출
ratio = 0.75
good_matches = [first for first, second in matches
               if first.distance < second.distance * ratio]
print(f'good matches:{len(good_matches):d}/{len(matches):d}')

good matches:23/500


In [10]:
# 좋은 매칭점의 queryIdx로 원본 영상의 좌표 구하기
src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches])

In [11]:
# 좋은 매칭점의 trainIdx로 대상 영역의 좌표 구하기
dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches])

In [12]:
# 원근 변환행렬 구하기
mtrx, mask = cv2.findHomography(src_pts, dst_pts)

In [14]:
# 원본 영상 크기로 변환 영역 좌표 생성
h, w, = img1.shape[:2]
pts = np.float32([ [[0, 0]], [[0, h-1]], [[w-1, h-1]], [[w-1, 0]] ])

In [15]:
# 원본 영상 좌표를 원근 변환
dst = cv2.perspectiveTransform(pts, mtrx)

In [16]:
# 변환 좌표 영역을 대상 영상에 그리기
img2 = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

In [17]:
# 좋은 매칭점을 그려서 출력
res = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None,
                     flags = cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

In [18]:
# 결과 출력
cv2.imshow('Matching Homography', res)
cv2.waitKey()
cv2.destroyAllWindows()